# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,617.163168,0.962314,1234,618.125481,616.200854,1236.250962,1232.401708
6,118.420898,-0.784828,1234,117.636070,119.205726,235.272140,238.411452
7,102.279303,-0.325752,1234,101.953551,102.605055,203.907101,205.210110
8,973.752880,-0.156347,1234,973.596533,973.909227,1947.193066,1947.818453
9,306.465084,0.279362,1234,306.744446,306.185722,613.488893,612.371445
10,904.710102,-0.076897,1234,904.633205,904.786999,1809.266411,1809.573997
11,886.579638,-0.579262,1234,886.000376,887.158900,1772.000752,1774.317800
12,868.716895,-0.182012,1234,868.534883,868.898906,1737.069767,1737.797813
13,389.609784,0.511480,1234,390.121264,389.098304,780.242528,778.196608
14,803.116512,-0.232505,1234,802.884008,803.349017,1605.768016,1606.698034


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-618.125481,616.200854
6,-117.636070,119.205726
7,-101.953551,102.605055
8,-973.596533,973.909227
9,-306.744446,306.185722
10,-904.633205,904.786999
11,-886.000376,887.158900
12,-868.534883,868.898906
13,-390.121264,389.098304
14,-802.884008,803.349017


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-618.125481,616.200854
6,-117.636070,119.205726
7,-101.953551,102.605055
8,-973.596533,973.909227
9,-306.744446,306.185722
10,-904.633205,904.786999
11,-886.000376,887.158900
12,-868.534883,868.898906
13,-390.121264,389.098304
14,-802.884008,803.349017


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-618.12548119,  616.20085407],
       [-117.63607005,  119.20572585],
       [-101.95355074,  102.6050552 ],
       [-973.59653275,  973.90922653],
       [-306.74444643,  306.18572245],
       [-904.63320527,  904.78699861],
       [-886.00037579,  887.15889993],
       [-868.53488336,  868.89890638],
       [-390.12126388,  389.09830381],
       [-802.88400777,  803.34901688]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1234.250962,-1232.401708
6,233.272140,-238.411452
7,201.907101,-205.210110
8,1945.193066,-1947.818453
9,611.488893,-612.371445
10,1807.266411,-1809.573997
11,1770.000752,-1774.317800
12,1735.069767,-1737.797813
13,778.242528,-778.196608
14,1603.768016,-1606.698034


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.617163,0.981157
6,0.118421,0.107586
7,0.102279,0.337124
8,0.973753,0.421827
9,0.306465,0.639681
10,0.904710,0.461552
11,0.886580,0.210369
12,0.868717,0.408994
13,0.389610,0.755740
14,0.803117,0.383748


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,617.163168,0.962314
6,118.420898,-0.784828
7,102.279303,-0.325752
8,973.752880,-0.156347
9,306.465084,0.279362
10,904.710102,-0.076897
11,886.579638,-0.579262
12,868.716895,-0.182012
13,389.609784,0.511480
14,803.116512,-0.232505


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
